# Init

In [1]:
# # if used pip install package
# !pip install xgboost
# !pip install lightgbm
# !pip install wget
# !pip install gensim
# !pip install catboost
# !pip install cython

In [2]:
from __future__ import print_function
import os
import sys
import logging
import gc
import wget
import time
import tarfile
import zipfile
import functools
import random
import copy
import pickle
from tqdm import tqdm_notebook, tqdm
from itertools import product, combinations
from scipy.special import comb, perm

import scipy
import numpy as np
import pandas as pd
import catboost as cbt
from glove import *

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.linear_model import LogisticRegression,BayesianRidge,SGDClassifier,PassiveAggressiveClassifier,RidgeClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC,NuSVC,SVC
from sklearn.metrics import f1_score, roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Embedding, Dropout, Activation, Conv1D, LSTM, GRU  #, CuDNNGRU, CuDNNLSTM
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import concatenate
from keras.callbacks import *
from keras.optimizers import *
from keras.engine.topology import Layer
from keras.layers import *
from keras.utils import np_utils

import gensim
import xgboost as xgb
import lightgbm as lgb

from joblib import Parallel, delayed

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Using TensorFlow backend.


In [3]:
# import ray
# ray.init(object_store_memory=int(100e6))
# import modin.pandas as pd
# os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
# os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask

In [4]:
class Logger(object):
    def __init__(self, filename='default.log', stream=sys.stdout):
        self.terminal = stream
        self.log = open(filename, 'a')
        
    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)
        
    def flush(self):
        pass

In [5]:
# sys.stdout = Logger("logs/default.log", sys.stdout)
# sys.stderr = Logger("logs/default_err.log", sys.stderr)

In [6]:
DDIR = "data"
UDDIR = "user_data"
UFEDIR = "user_data/feat_data_v05"
UMDIR = "user_data/model_data"
RESDIR = "prediction_result"

In [7]:
UID = "user_id"

# Load data (Only once)

In [8]:
# train_fname = wget.download("https://tesla-ap-shanghai-1256322946.cos.ap-shanghai.myqcloud.com/cephfs/tesla_common/deeplearning/dataset/algo_contest/train_preliminary.zip", out=DDIR)
# test_fname = wget.download("https://tesla-ap-shanghai-1256322946.cos.ap-shanghai.myqcloud.com/cephfs/tesla_common/deeplearning/dataset/algo_contest/test.zip", out=DDIR)

In [9]:
# def myunzip(filename):
#     zFile = zipfile.ZipFile(filename, "r")
#     for fileM in zFile.namelist(): 
#         zFile.extract(fileM, DDIR)
#         print(fileM)
#     zFile.close()

In [10]:
# myunzip(train_fname)
# myunzip(test_fname)

# Utils

In [11]:
def bch_rencol(values, prefix="", suffix=""):
    return list(map(lambda x: f"{prefix}"+"_".join(list(map(lambda y: str(y), x)))+f"{suffix}" 
                    if hasattr(x, "__iter__") and not isinstance(x, str) 
                    else f"{prefix}"+str(x)+f"{suffix}", values))

In [12]:
def mynunique(values):
    return values.nunique(dropna=False)
def getidxmax(x):
    return x.idxmax()[1]
# for time series
def at_len(x):
    return len(x)

def at_sum(x):
    return np.sum(x)

def at_max(x):
    return np.max(x)

def at_min(x):
    return np.min(x)

def at_mean(x):
    return np.mean(x)

def at_range(x):
    return at_max(x) - at_min(x)

def at_nunq(x):
    return len(set(x))

def at_lenDrange(x):
    return at_len(x)/(at_range(x)+1)

def at_lenDnunq(x):
    return at_len(x)/at_nunq(x)

def at_percentile(n):
    def at_percentile_(x):
        return np.percentile(x, n)
    at_percentile_.__name__ = f"at_percentile_{n}"
    return at_percentile_

In [13]:
OP_SET = ["sum", "max", "min", "mean", "std"]
OP_SET1 = ["nunique", "sum", "max", "min", "mean", "std", "median", "skew", at_percentile(25), at_percentile(75)]
OP_SET2 = ["sum", "max", "min", "mean", "std", "median", "skew", at_percentile(25), at_percentile(75)]

In [14]:
ID_SET = ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry", "time", "click_times"]

In [15]:
nesting_level = 0
is_start = None
class Timer:
    def __init__(self):
        self.start = time.time()
        self.history = [self.start]

    def check(self, info):
        current = time.time()
        print(f"[{info}] spend {current - self.history[-1]:0.2f} sec")
        self.history.append(current)

def log(entry):
    global nesting_level
    space = "-" * (4 * nesting_level)
    print(f"{space}{entry}")

def timeit(method, start_log=None):
    @functools.wraps(method)
    def timed(*args, **kw):
        global is_start
        global nesting_level

        if not is_start:
            print()

        is_start = True
        log(f"Start [{method.__name__}]:" + (start_log if start_log else ""))
        log(f'Start time: {time.strftime("%Y-%m-%d %H:%M:%S")}')
        nesting_level += 1

        start_time = time.time()
        result = method(*args, **kw)
        end_time = time.time()

        nesting_level -= 1
        log(f"End   [{method.__name__}]. Time elapsed: {end_time - start_time:0.2f} sec.")
        is_start = False

        return result

    return timed

In [16]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Stats features

In [17]:
# read train data
train_click_log = pd.read_csv(f"{DDIR}/train_preliminary/click_log.csv")
train_ad = pd.read_csv(f"{DDIR}/train_preliminary/ad.csv")
# tag
train_user = pd.read_csv(f"{DDIR}/train_preliminary/user.csv")

In [18]:
# read test data
test_click_log = pd.read_csv(f"{DDIR}/test/click_log.csv")
test_ad = pd.read_csv(f"{DDIR}/test/ad.csv")
# pd.DataFrame(np.sort(test_click_log[UID].unique()), columns=[UID]).to_csv(f"{DDIR}/test/user.csv", index=False)
test_user = pd.read_csv(f"{DDIR}/test/user.csv")

In [19]:
train_click_log.shape

(30082771, 4)

In [20]:
train_ad.shape

(2481135, 6)

In [21]:
train_user.shape

(900000, 3)

In [22]:
test_click_log.shape

(33585512, 4)

In [23]:
test_ad.shape

(2618159, 6)

In [24]:
train_ad["product_id"] = train_ad["product_id"].replace("\\N", "0").astype(int)
train_ad["industry"] = train_ad["industry"].replace("\\N", "0").astype(int)

In [25]:
test_ad["product_id"] = test_ad["product_id"].replace("\\N", "0").astype(int)
test_ad["industry"] = test_ad["industry"].replace("\\N", "0").astype(int)

In [26]:
# creative id in train (creative id is unique in train_ad)
len(train_ad)

2481135

In [27]:
# creative id in test (creative id is unique in test_ad)
len(test_ad)

2618159

In [28]:
tol_train = pd.merge(train_click_log, train_ad, how="left", on="creative_id")
tol_test = pd.merge(test_click_log, test_ad, how="left", on="creative_id")

In [29]:
del train_click_log, train_ad
del test_click_log, test_ad

In [ ]:
# comb pairs
# ad_id-product_id
# product_id-product_category
# ad_id-advertiser_id
# advertiser_id-industry
# product_category-industry
# product_category-advertiser_id
comb_set = ["ad_id-product_id", "product_id-product_category", "ad_id-advertiser_id", 
            "advertiser_id-industry", "product_category-industry", "product_category-advertiser_id",
           ]

In [ ]:
# tol_train["ad_id-product_id"] = list(map(lambda x, y: f"{x}_{y}", tol_train["ad_id"], tol_train["product_id"]))
# tol_train["product_id-product_category"] = list(map(lambda x, y: f"{x}_{y}", tol_train["product_id"], tol_train["product_category"]))
# tol_train["ad_id-advertiser_id"] = list(map(lambda x, y: f"{x}_{y}", tol_train["ad_id"], tol_train["advertiser_id"]))
# tol_train["advertiser_id-industry"] = list(map(lambda x, y: f"{x}_{y}", tol_train["advertiser_id"], tol_train["industry"]))
# tol_train["product_category-industry"] = list(map(lambda x, y: f"{x}_{y}", tol_train["product_category"], tol_train["industry"]))
# tol_train["product_category-advertiser_id"] = list(map(lambda x, y: f"{x}_{y}", tol_train["product_category"], tol_train["advertiser_id"]))

In [ ]:
# tol_test["ad_id-product_id"] = list(map(lambda x, y: f"{x}_{y}", tol_test["ad_id"], tol_test["product_id"]))
# tol_test["product_id-product_category"] = list(map(lambda x, y: f"{x}_{y}", tol_test["product_id"], tol_test["product_category"]))
# tol_test["ad_id-advertiser_id"] = list(map(lambda x, y: f"{x}_{y}", tol_test["ad_id"], tol_test["advertiser_id"]))
# tol_test["advertiser_id-industry"] = list(map(lambda x, y: f"{x}_{y}", tol_test["advertiser_id"], tol_test["industry"]))
# tol_test["product_category-industry"] = list(map(lambda x, y: f"{x}_{y}", tol_test["product_category"], tol_test["industry"]))
# tol_test["product_category-advertiser_id"] = list(map(lambda x, y: f"{x}_{y}", tol_test["product_category"], tol_test["advertiser_id"]))

## Category map to Label (use KFold)

__a little overfitting age__

In [ ]:

# two kfold
@timeit
def gen_catemlb(train_data, test_data, tag, col):
    train_use = pd.merge(train_data[[UID, col]], tag, how="left", on=UID)
    test_use = test_data[[UID, col]]
    
    nfolds = 5
    kfold = KFold(n_splits=5, shuffle=False, random_state=2020)
    
    kf_map = dict()

    re_train_use = pd.DataFrame()
    re_test_use = pd.DataFrame()

    for kfn, (tr_idx, val_idx) in enumerate(kfold.split(train_use)):
        print("train current kfn: ", kfn)
        tr_ucid, val_ucid = train_use.iloc[tr_idx], train_use.iloc[val_idx]
        cur_kf_map = tr_ucid.groupby([col])[["age", "gender"]].agg(["mean"])
        cur_kf_map.columns = bch_rencol(cur_kf_map.columns, prefix=f"{col}_")
        # # only use intersect between train and test
        # cur_kf_map.drop(np.setdiff1d(kf_map.index.unique(), test_use[col].unique()), inplace=True)
        kf_map[kfn] = copy.deepcopy(cur_kf_map)
        
        val_ucid = pd.merge(val_ucid, cur_kf_map, how="left", on=col)
        re_train_use = pd.concat([re_train_use, val_ucid])
    
    
    for kfn, (_, val_idx) in enumerate(kfold.split(test_use)):
        print("test current kfn: ", kfn)
        val_ucid = test_use.iloc[val_idx]
        val_ucid = pd.merge(val_ucid, kf_map[kfn], how="left", on=col)
        re_test_use = pd.concat([re_test_use, val_ucid])

    
    assert len(train_use) == len(re_train_use)
    assert len(test_use) == len(re_test_use)

    train_use = None
    test_use = None

    tmp = re_train_use.groupby([UID], sort=False)[f"{col}_age_mean", f"{col}_gender_mean"].agg(["sum", "max", "mean", "median", "min", "std", ])
    tmp.columns = bch_rencol(tmp.columns)
    tmp.to_pickle(f"{UFEDIR}/train_stats_catemlb_{col}.pkl")
    tmp = None

    tmp = re_test_use.groupby([UID], sort=False)[f"{col}_age_mean", f"{col}_gender_mean"].agg(["sum", "max", "mean", "median", "min", "std", ])
    tmp.columns = bch_rencol(tmp.columns)
    tmp.to_pickle(f"{UFEDIR}/test_stats_catemlb_{col}.pkl")
    tmp = None

In [ ]:
for col in ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]:
    gen_catemlb(tol_train, tol_test, train_user, col)

## One key(O1)

In [30]:
# for train
tmp = tol_train.groupby([UID], sort=False).agg(
    {
        UID: ["count"], 
        "click_times": OP_SET1,
        "time": OP_SET1,
        "creative_id": ["nunique"],
        "ad_id": ["nunique"],
        "product_id": ["nunique"],
        "product_category": ["nunique"],
        "advertiser_id": ["nunique"],
        "industry": ["nunique"],
    }
)

tmp.columns = bch_rencol(tmp.columns)

tmp.to_pickle(f"{UFEDIR}/train_stats_o1.pkl")
tmp = None

In [31]:
# for test
tmp = tol_test.groupby([UID], sort=False).agg(
    {
        UID: ["count"], 
        "click_times": OP_SET1,
        "time": OP_SET1,
        "creative_id": ["nunique"],
        "ad_id": ["nunique"],
        "product_id": ["nunique"],
        "product_category": ["nunique"],
        "advertiser_id": ["nunique"],
        "industry": ["nunique"],
    }
)

tmp.columns = bch_rencol(tmp.columns)

tmp.to_pickle(f"{UFEDIR}/test_stats_o1.pkl")
tmp = None

In [ ]:
gc.collect()

## Comb Key(O2)

In [ ]:
id_list = ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]

In [ ]:
# train
for i in tqdm(range(1, 7)):
    for cl in combinations(id_list, i):
        key_set = [UID] + list(cl)
        tmp = tol_train.groupby(key_set, sort=False)[key_set[-1:]].agg(["count"]).groupby([UID]).agg(OP_SET2)
        tmp.columns = bch_rencol(tmp.columns, prefix=f"{'C'.join(key_set[1:-1])}_")
        tmp.to_pickle(f"{UFEDIR}/train_stats_o2_{'C'.join(key_set)}.pkl")
        tmp = None

In [ ]:
# test
for i in tqdm(range(1, 7)):
    for cl in combinations(id_list, i):
        key_set = [UID] + list(cl)
        tmp = tol_test.groupby(key_set, sort=False)[key_set[-1:]].agg(["count"]).groupby([UID]).agg(OP_SET2)
        tmp.columns = bch_rencol(tmp.columns, prefix=f"{'C'.join(key_set[1:-1])}_")
        tmp.to_pickle(f"{UFEDIR}/test_stats_o2_{'C'.join(key_set)}.pkl")
        tmp = None

In [ ]:
gc.collect()

## Time Windows-Time cost(not use)

In [ ]:
tm_tw = Timer()

### Time Bins

In [ ]:
bins_set = [3, 7, 10, 30]

for bins in bins_set:
    tol_bins = pd.cut(pd.concat([tol_train["time"], tol_test["time"]]), bins, labels=range(bins))
    tol_train[f"bins{bins}"] = tol_bins[:len(tol_train)]
    tol_test[f"bins{bins}"] = tol_bins[len(tol_train):]

In [ ]:
tm_tw.check("Init Bin Set")

#### One Key(O1)

In [ ]:
# For train
for bins in bins_set:
    for cb in tqdm(range(bins)):
        tmp = tol_train[tol_train[f"bins{bins}"] == cb].groupby([UID], sort=False).agg(
            {
                UID: ["count"], 
                "click_times": OP_SET1,
                "time": OP_SET1,
                "creative_id": ["nunique"],
                "ad_id": ["nunique"],
                "product_id": ["nunique"],
                "product_category": ["nunique"],
                "advertiser_id": ["nunique"],
                "industry": ["nunique"],
            }
        )
        tmp.columns = bch_rencol(tmp.columns, prefix=f"bins{bins}_{cb}_")
        tmp.to_pickle(f"{UFEDIR}/train_stats_o1_bins{bins}_{cb}.pkl")
        tmp = None

In [ ]:
tm_tw.check("Generate o1 for train")

In [ ]:
# For test
for bins in bins_set:
    for cb in tqdm(range(bins)):
        tmp = tol_test[tol_test[f"bins{bins}"] == cb].groupby([UID], sort=False).agg(
            {
                UID: ["count"], 
                "click_times": OP_SET1,
                "time": OP_SET1,
                "creative_id": ["nunique"],
                "ad_id": ["nunique"],
                "product_id": ["nunique"],
                "product_category": ["nunique"],
                "advertiser_id": ["nunique"],
                "industry": ["nunique"],
            }
        )
        tmp.columns = bch_rencol(tmp.columns, prefix=f"bins{bins}_{cb}_")
        tmp.to_pickle(f"{UFEDIR}/test_stats_o1_bins{bins}_{cb}.pkl")
        tmp = None

In [ ]:
tm_tw.check("Generate o1 for test")

In [ ]:
gc.collect()

#### Comb Key(O2)

In [ ]:
id_list = ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]

In [ ]:
# train
for bins in bins_set:
    for cb in tqdm(range(bins)):
        for i in tqdm(range(1, 7)):
            for cl in combinations(id_list, i):
                key_set = [UID] + list(cl)
                tmp = tol_train[tol_train[f"bins{bins}"] == cb].groupby(key_set, sort=False)[key_set[-1:]].agg(["count"]).groupby([UID]).agg(OP_SET2)
                tmp.columns = bch_rencol(tmp.columns, prefix=f"bins{bins}_{cb}_{'C'.join(key_set[1:-1])}_")
                tmp.to_pickle(f"{UFEDIR}/train_stats_o2_bins{bins}_{cb}_{'C'.join(key_set)}.pkl")
                tmp = None

In [ ]:
tm_tw.check("Generate o2 bin for train")

In [ ]:
# test
for bins in bins_set:
    for cb in tqdm(range(bins)):
        for i in tqdm(range(1, 7)):
            for cl in combinations(id_list, i):
                key_set = [UID] + list(cl)
                tmp = tol_test[tol_test[f"bins{bins}"] == cb].groupby(key_set, sort=False)[key_set[-1:]].agg(["count"]).groupby([UID]).agg(OP_SET2)
                tmp.columns = bch_rencol(tmp.columns, prefix=f"bins{bins}_{cb}_{'C'.join(key_set[1:-1])}_")
                tmp.to_pickle(f"{UFEDIR}/test_stats_o2_bins{bins}_{cb}_{'C'.join(key_set)}.pkl")
                tmp = None

In [ ]:
tm_tw.check("Generate o2 bin for test")

### Time Slides

In [ ]:
slide_set = [1, 2, 3, 7, 14, 21, 30, 60, 90]

tol_train["max_time"] = tol_train.groupby([UID], sort=False)["time"].transform("max")
tol_test["max_time"] = tol_test.groupby([UID], sort=False)["time"].transform("max")

In [ ]:
tm_tw.check("Get Max time")

#### One Key(O1)

In [ ]:
# For train
for win in slide_set:
    tmp = tol_train[tol_train["time"] >= tol_train["max_time"] - win].groupby([UID], sort=False).agg(
        {
            UID: ["count"], 
            "click_times": OP_SET1,
            "time": OP_SET1,
            "creative_id": ["nunique"],
            "ad_id": ["nunique"],
            "product_id": ["nunique"],
            "product_category": ["nunique"],
            "advertiser_id": ["nunique"],
            "industry": ["nunique"],
        }
    )
    tmp.columns = bch_rencol(tmp.columns, prefix=f"slide{win}_")
    tmp.to_pickle(f"{UFEDIR}/train_stats_o1_slide{win}.pkl")
    tmp = None

In [ ]:
tm_tw.check("Generate o1 slide for train")

In [ ]:
# For test
for win in slide_set:
    tmp = tol_test[tol_test["time"] >= tol_test["max_time"] - win].groupby([UID], sort=False).agg(
        {
            UID: ["count"], 
            "click_times": OP_SET1,
            "time": OP_SET1,
            "creative_id": ["nunique"],
            "ad_id": ["nunique"],
            "product_id": ["nunique"],
            "product_category": ["nunique"],
            "advertiser_id": ["nunique"],
            "industry": ["nunique"],
        }
    )
    tmp.columns = bch_rencol(tmp.columns, prefix=f"slide{win}_")
    tmp.to_pickle(f"{UFEDIR}/test_stats_o1_slide{win}.pkl")
    tmp = None

In [ ]:
tm_tw.check("Generate o1 slide for test")

#### Comb Key(O2)

In [ ]:
id_list = ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]

In [ ]:
# train
for win in slide_set:
    for i in tqdm(range(1, 7)):
        for cl in combinations(id_list, i):
            key_set = [UID] + list(cl)
            tmp = tol_train[tol_train["time"] >= tol_train["max_time"] - win].groupby(key_set, sort=False)[key_set[-1:]].agg(["count"]).groupby([UID]).agg(OP_SET2)
            tmp.columns = bch_rencol(tmp.columns, prefix=f"slide{win}_{'C'.join(key_set[1:-1])}_")
            tmp.to_pickle(f"{UFEDIR}/train_stats_o2_slide{win}_{'C'.join(key_set)}.pkl")
            tmp = None

In [ ]:
tm_tw.check("Generate o2 slide for train")

In [ ]:
# test
for win in slide_set:
    for i in tqdm(range(1, 7)):
        for cl in combinations(id_list, i):
            key_set = [UID] + list(cl)
            tmp = tol_test[tol_test["time"] >= tol_test["max_time"] - win].groupby(key_set, sort=False)[key_set[-1:]].agg(["count"]).groupby([UID]).agg(OP_SET2)
            tmp.columns = bch_rencol(tmp.columns, prefix=f"slide{win}_{'C'.join(key_set[1:-1])}_")
            tmp.to_pickle(f"{UFEDIR}/test_stats_o2_slide{win}_{'C'.join(key_set)}.pkl")
            tmp = None

In [ ]:
tm_tw.check("Generate o2 slide for test")

In [ ]:
gc.collect()

## (TOP Category) One-Hot

In [ ]:
tol_data = pd.concat([tol_train, tol_test])

In [ ]:
dum_dic = {
    "creative_id": 300,
    "ad_id": 300,
    "product_id": 100,
    "advertiser_id": 100,
}

In [ ]:
for cid in dum_dic:
    use_id = tol_data[cid].value_counts().head(dum_dic[cid]).index.values
    tol_data[f"{cid}_dummy"] = tol_data[cid]
    tol_data[f"{cid}_dummy"][~tol_data[f"{cid}_dummy"].isin(use_id)] = -999

In [ ]:
for cid in  ["time", "click_times", "creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]:
    ocid = cid
    if cid in dum_dic:
        cid = f"{cid}_dummy"
    tmp = tol_data[[UID, cid]].groupby([UID, cid], sort=False)[[cid]].agg(["count"]).unstack().fillna(0)
    tmp.columns = bch_rencol(tmp.columns)
    reduce_mem_usage(tmp).to_pickle(f"{UFEDIR}/onehot_catecnt_{ocid}.pkl")
    tmp = None

In [ ]:
gc.collect()

## Time diff (TODO)